In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [ ]:
!pip install mlxtend

In [3]:
import os
os.chdir("/content/drive/MyDrive/CMPT733_Final_Project")


In [4]:
files = [file for file in os.listdir("tweet_sentiment_added_noise_removed") if "tag_#" in file]

In [5]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

files = [file for file in os.listdir("tweet_sentiment_added_noise_removed") if "tag_#" in file]

# files = ['tag_#lockdown.csv']

hashtags = ['coronavirus', 'covid', 'covid19', 'covid-19',
           'sarscov2', 'covid_19', '#ncov', 'ncov2019', '2019-ncov',
           'pandemic', '2019ncov', 'quarantine', 'flatteningthecurve',
           'flattenthecurve', 'handsanitizer', 'lockdown',
           'socialdistancing', 'workfromhome', 'workingfromhome', 'ppe',
           'n95', 'covidiots', 'herdimmunity', 'pneumonia',
           'wearamask', 'vaccine', 'vaccines', 'coronavaccine', 'coronavaccines',
           'faceshield', 'faceshields', 'health worker', 'healthworkers',
           'stayhomestaysafe', 'coronaupdate', 'frontlineheroes',
           'coronawarriors', 'homeschool', 'homeschooling', 'hometasking',
           'masks4all', 'wfh', 'washurhands', 'washyourhands',
           'stayathome', 'stayhome', 'selfisolating']

def extract_tags(Text):
  tags = re.findall(r"#(\w+)", Text)
  return tags

df = pd.read_csv('./tweet_sentiment_added_noise_removed/tag_#coronavirus.csv', lineterminator='\n')

for file in files[1:]: 
  df_append = pd.read_csv('./tweet_sentiment_added_noise_removed/' + file, lineterminator='\n')
  df = df.append(df_append, ignore_index = True)

df['Tags'] = df['Text'].apply(extract_tags)
tags_list = df['Tags'].to_list()

tags_filter_list = []
for sublist in tags_list:
  sublist = [i for i in sublist if i in hashtags]
  if sublist:
    tags_filter_list.append(sublist)
    
  
encoder = TransactionEncoder()

tags_array_encoded = encoder.fit(tags_filter_list).transform(tags_filter_list)

tags_df_encoded = pd.DataFrame(tags_array_encoded, columns=encoder.columns_)

tags_cluster = apriori(tags_df_encoded, min_support=0.001, use_colnames=True)

tags_cluster


,support,itemsets
0,0.001180,(coronaupdate)
1,0.003762,(coronavaccine)
2,0.347352,(coronavirus)
3,0.015945,(covid)
4,0.074533,(covid19)
...,...,...
87,0.001389,"(covid19, coronavirus, socialdistancing)"
88,0.001063,"(covid19, coronavirus, stayhome)"
89,0.002165,"(pandemic, lockdown, coronavirus)"
90,0.001102,"(pandemic, vaccine, coronavirus)"


In [6]:
# remove tags contains only 1 item
tags_cluster['length'] = tags_cluster['itemsets'].apply(lambda x: len(x))
tags_cluster_filtered = tags_cluster[(tags_cluster['length'] >= 2)].sort_values(by=['support'], ascending=False)
tag_list = [list(i) for i in tags_cluster_filtered.itemsets]
tag_list

[['pandemic', 'coronavirus'],
 ['covid19', 'coronavirus'],
 ['lockdown', 'coronavirus'],
 ['pandemic', 'covid19'],
 ['pandemic', 'lockdown'],
 ['vaccine', 'coronavirus'],
 ['covid19', 'lockdown'],
 ['coronavirus', 'covid'],
 ['quarantine', 'coronavirus'],
 ['quarantine', 'lockdown'],
 ['coronavirus', 'socialdistancing'],
 ['pandemic', 'covid19', 'coronavirus'],
 ['pandemic', 'covid'],
 ['quarantine', 'covid19'],
 ['covid19', 'socialdistancing'],
 ['washyourhands', 'wearamask'],
 ['quarantine', 'stayhome'],
 ['lockdown', 'covid'],
 ['pandemic', 'vaccine'],
 ['wfh', 'workfromhome'],
 ['covid19', 'covid'],
 ['pandemic', 'quarantine'],
 ['coronavirus', 'stayhome'],
 ['lockdown', 'socialdistancing'],
 ['covid19', 'stayhome'],
 ['lockdown', 'stayhome'],
 ['pandemic', 'lockdown', 'coronavirus'],
 ['socialdistancing', 'stayhome'],
 ['pandemic', 'socialdistancing'],
 ['covid19', 'lockdown', 'coronavirus'],
 ['quarantine', 'socialdistancing'],
 ['covid19', 'wearamask'],
 ['quarantine', 'covid'],

In [26]:
# find related tags
target_tag = 'vaccine'
related_tags = []
for tags in tag_list:
  if target_tag in tags:
    related_tags.extend(tags)
related_tags = list(set(related_tags))
related_tags.remove(target_tag)
related_tags = [ '#'+tag for tag in related_tags]
related_tags

['#lockdown', '#pandemic', '#covid19', '#coronavirus']

In [8]:
# get all values

colnames = ['date']
daily_mean_df = pd.DataFrame(columns=['date'])
weekly_mean_df = pd.DataFrame(columns=['date'])
monthly_mean_df = pd.DataFrame(columns=['date'])


for file in files: 
    df = pd.read_csv('./tweet_sentiment_added_noise_removed/' + file, lineterminator='\n')
    tag = re.sub('.csv', '', re.sub('tag_', '', file))
    colnames.append(tag)

    df['compound'] = np.where(df['compound'] >= 0.05, 1, df['compound'])
    df['compound'] = np.where(df['compound'] <= -0.05, -1, df['compound']) 
    df['compound'] = np.where(df['compound'].between(-0.05, 0.05), 0, df['compound'])
    
    mean_df = df.groupby("date")['compound'].mean()
    mean_df.index = pd.to_datetime(mean_df.index)

    daily_mean_df = daily_mean_df.merge(mean_df, how='outer', on='date')
    weekly_mean_df = weekly_mean_df.merge(mean_df.resample("W").mean(), how='outer', on='date')
    monthly_mean_df = monthly_mean_df.merge(mean_df.resample("M").mean(), how='outer', on='date')

daily_mean_df.columns = colnames
weekly_mean_df.columns = colnames
monthly_mean_df.columns = colnames

In [12]:
weekly_mean_df.columns

Index(['date', '#coronavirus', '#coronaupdate', '#selfisolating',
       '#quarantine', '#wearamask', '#stayhomestaysafe', '#pneumonia',
       '#herdimmunity', '#wfh', '#masks4all', '#covid19', '#faceshield',
       '#sarscov2', '#frontlineheroes', '#washyourhands', '#covid-19',
       '#coronavaccine', '#flattenthecurve', '#covidvaccine', '#workfromhome',
       '#socialbubble', '#ppe', '#socialdistancing', '#lockdown', '#pandemic'],
      dtype='object')

In [20]:
# predict target tag value
column_selected = related_tags + ['date']
predicted_df = weekly_mean_df[column_selected]
predicted_df[target_tag] = weekly_mean_df[related_tags].mean(axis=1)

df = predicted_df.melt(id_vars="date", var_name="hashtag", value_name="compound_score")
fig = px.line(df, x = "date", y = "compound_score", color = "hashtag", height=800, title='Weekly Mean Compound Sentiment Score')
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
predicted_df[target_tag + '_groundtruth'] = weekly_mean_df['#covidvaccine']
predicted_df = predicted_df[['vaccine_groundtruth', 'vaccine','date']]
df = predicted_df.melt(id_vars="date", var_name="hashtag", value_name="compound_score")
fig = px.line(df, x = "date", y = "compound_score", color = "hashtag", height=800, title='Weekly Mean Compound Sentiment Score')
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

